<a href="https://colab.research.google.com/github/Dcodinginsane/Data-Analysis-project/blob/main/Customer_Segmentation_Kmeans_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Segmentation with K-Means Clustering

Customer segmentation will be applied to an e-commerce customer database using K-means clustering from scikit-learn. It is an [extension](https://github.com/cereniyim/Data-Science-Projects/blob/master/ECommerce-Sales-Data-EDA/ECommerce_Sales_Data_Analysis.ipynb) of a case study solved couple of months ago.

The provided customers database is visualized as part of a case study. This project is taking the case study one step further with the following motive:

**Can this customer database be grouped to develop customized relationships?**

**To answer this question 3 features will be created and used:** <br>
- products ordered
- average return rate
- total spending

**Dataset represents real customers & orders data between November 2018 - April 2019 and it is pseudonymized for confidentiality.**

**Imports**

In [1]:
# data wrangling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for data preprocessing and clustering
from sklearn.cluster import KMeans

%matplotlib inline
# to include graphs inline within the frontends next to code

%config InlineBackend.figure_format='retina'
#to enable retina (high resolution) plots

pd.options.mode.chained_assignment = None
# to bypass warnings in various dataframe assignments

**Investigate data**

In [2]:
# load data into a dataframe
customers_orders = pd.read_csv("Orders - Analysis Task.csv")

In [3]:
# first rows of the dataset
customers_orders.head()

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
0,DPR,DPR,100,AD-982-708-895-F-6C894FB,52039657.0,1312378.0,8.329072e+13,04/12/2018,2.0,200.0,-200.00,0.00,0.0,0.0,0.0,0.0,2.0
1,RJF,Product P,28 / A / MTM,83-490-E49-8C8-8-3B100BC,56914686.0,3715657.0,3.625379e+13,01/04/2019,2.0,190.0,-190.00,0.00,0.0,0.0,0.0,0.0,2.0
2,CLH,Product B,32 / B / FtO,68-ECA-BC7-3B2-A-E73DE1B,24064862.0,9533448.0,7.309456e+13,05/11/2018,0.0,164.8,-156.56,-8.24,0.0,0.0,0.0,-2.0,2.0
3,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868.0,4121004.0,5.361658e+13,19/02/2019,1.0,119.0,-119.00,0.00,0.0,0.0,0.0,0.0,1.0
4,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868.0,4121004.0,2.926322e+13,19/02/2019,1.0,119.0,-119.00,0.00,0.0,0.0,0.0,0.0,1.0


In [4]:
# first glance of customers_orders data
customers_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31200 entries, 0 to 31199
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_title           31200 non-null  object 
 1   product_type            31199 non-null  object 
 2   variant_title           31199 non-null  object 
 3   variant_sku             31199 non-null  object 
 4   variant_id              31199 non-null  float64
 5   customer_id             31199 non-null  float64
 6   order_id                31199 non-null  float64
 7   day                     31199 non-null  object 
 8   net_quantity            31199 non-null  float64
 9   gross_sales             31199 non-null  float64
 10  discounts               31199 non-null  float64
 11  returns                 31199 non-null  float64
 12  net_sales               31199 non-null  float64
 13  taxes                   31199 non-null  float64
 14  total_sales             31199 non-null

In [5]:
# descriptive statistics of the non-object columns
customers_orders.describe()

,variant_id,customer_id,order_id,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
count,3.119900e+04,3.119900e+04,3.119900e+04,31199.000000,31199.000000,31199.000000,31199.000000,31199.000000,31199.000000,31199.000000,31199.000000,31199.000000
mean,2.095412e+11,5.531078e+11,5.510620e+13,1.019937,80.290890,-11.114159,-0.033386,69.143345,13.578258,82.721603,-0.000737,1.020674
std,3.907170e+12,6.044965e+12,2.582070e+13,0.160493,18.902017,8.188143,1.566940,20.982975,4.333352,24.753848,0.029409,0.158490
min,1.001447e+07,1.000661e+06,1.000657e+13,0.000000,24.160000,-200.000000,-142.500000,0.000000,0.000000,0.000000,-2.000000,1.000000
25%,2.692223e+07,3.288240e+06,3.277868e+13,1.000000,74.170000,-16.500000,0.000000,58.330000,11.670000,70.000000,0.000000,1.000000
50%,4.494514e+07,5.525012e+06,5.517150e+13,1.000000,79.170000,-10.750000,0.000000,66.000000,13.040000,79.200000,0.000000,1.000000
75%,7.443784e+07,7.792786e+06,7.733663e+13,1.000000,82.500000,-5.500000,0.000000,80.100000,14.965000,95.200000,0.000000,1.000000
max,8.422212e+13,9.954992e+13,9.999554e+13,6.000000,445.000000,0.000000,0.000000,445.000000,63.340000,445.000000,0.000000,6.000000


There were significant number of rows whose `ordered_item_quantity` is 0 and `net_quantity` is less than 0, which means they are not ordered/sold at all; but the fact that they have returns requires investigation.

In [6]:
print("Number of rows that net quantity is negative:",
      customers_orders[customers_orders.net_quantity < 0].shape[0])

Number of rows that net quantity is negative: 0


**These rows will be excluded from the orders dataset for the project.**

In [7]:
# exclude not sold/ordered SKUs from the dataset
customers_orders = customers_orders[
    customers_orders["ordered_item_quantity"] > 0]

## 1. Products ordered
It is the count of the products ordered in product_type column by a customer. <br>

**Create functions to identify customers who order multiple products**

In [8]:
def encode_column(column):
    if column > 0:
        return 1
    if column <= 0:
        return 0


def aggregate_by_ordered_quantity(dataframe, column_list):
    '''this function:
    1. aggregates a given dataframe by column list,
    as a result creates a aggregated dataframe by counting the ordered item quantities

    2. adds number_of_X ordered where X is the second element in the column_list
    to the aggregated dataframe by encoding ordered items into 1

    3. creates final dataframe containing information about
    how many of X are ordered, based on the first element passed in the column list'''

    aggregated_dataframe = (dataframe
                            .groupby(column_list)
                            .ordered_item_quantity.count()
                            .reset_index())

    aggregated_dataframe["products_ordered"] = (aggregated_dataframe
                                                 .ordered_item_quantity
                                                 .apply(encode_column))

    final_dataframe = (aggregated_dataframe
                       .groupby(column_list[0])
                       .products_ordered.sum() # aligned with the added column name
                       .reset_index())

    return final_dataframe

In [9]:
# apply functions to customers_orders
customers = aggregate_by_ordered_quantity(customers_orders, ["customer_id", "product_type"])

In [10]:
print(customers.head())

   customer_id  products_ordered
0    1000661.0                 1
1    1001914.0                 1
2    1002167.0                 1
3    1002419.0                 2
4    1003002.0                 1


## 2. Average Return Rate
It is the ratio of returned item quantity and ordered item quantity. This ratio is first calculated per order and then averaged for all orders of a customer.

In [11]:
# aggregate data per customer_id and order_id,
# to see ordered item sum and returned item sum
ordered_sum_by_customer_order = (customers_orders
                                 .groupby(["customer_id", "order_id"])
                                 .ordered_item_quantity.sum()
                                 .reset_index())

returned_sum_by_customer_order = (customers_orders
                                  .groupby(["customer_id", "order_id"])
                                  .returned_item_quantity.sum()
                                  .reset_index())

# merge two dataframes to be able to calculate unit return rate
ordered_returned_sums = pd.merge(ordered_sum_by_customer_order, returned_sum_by_customer_order)

In [12]:
# calculate unit return rate per order and customer
ordered_returned_sums["average_return_rate"] = (-1 *
                                             ordered_returned_sums["returned_item_quantity"] /
                                             ordered_returned_sums["ordered_item_quantity"])

In [13]:
ordered_returned_sums.head()

,customer_id,order_id,ordered_item_quantity,returned_item_quantity,average_return_rate
0,1000661.0,9.911999e+13,3.0,0.0,-0.0
1,1001914.0,7.975857e+13,1.0,0.0,-0.0
2,1002167.0,5.744015e+13,1.0,0.0,-0.0
3,1002419.0,9.112338e+13,2.0,0.0,-0.0
4,1003002.0,8.788005e+13,2.0,0.0,-0.0


In [14]:
# take average of the unit return rate for all orders of a customer
customer_return_rate = (ordered_returned_sums
                        .groupby("customer_id")
                        .average_return_rate
                        .mean()
                        .reset_index())

In [15]:
return_rates = pd.DataFrame(customer_return_rate["average_return_rate"]
                            .value_counts()
                            .reset_index())

return_rates.rename(columns=
                    {"index": "average return rate",
                     "average_return_rate": "count of unit return rate"},
                    inplace=True)

return_rates.sort_values(by="average return rate")

,average return rate,count of unit return rate
0,0.000000,16494
4,0.111111,1
3,0.333333,2
2,0.500000,5
5,0.666667,1
1,1.000000,6


In [16]:
# add average_return_rate to customers dataframe
customers = pd.merge(customers,
                     customer_return_rate,
                     on="customer_id")

## 3. Total spending
Total spending is the aggregated sum of total sales value which is the amount after the taxes and returns.

In [17]:
# aggreagate total sales per customer id
customer_total_spending = (customers_orders
                           .groupby("customer_id")
                           .total_sales
                           .sum()
                           .reset_index())

customer_total_spending.rename(columns = {"total_sales" : "total_spending"},
                               inplace = True)

## Create features data frame

In [18]:
# add total sales to customers dataframe
customers = customers.merge(customer_total_spending,
                            on="customer_id")

In [19]:
print("The number of customers from the existing customer base:", customers.shape[0])

The number of customers from the existing customer base: 16509


In [20]:
# drop id column since it is not a feature
customers.drop(columns="customer_id",
               inplace=True)

In [21]:
customers.head()

,products_ordered,average_return_rate,total_spending
0,1,0.0,260.0
1,1,0.0,79.2
2,1,0.0,79.2
3,2,0.0,103.0
4,1,0.0,153.0


### Visualize features

In [22]:
fig = make_subplots(rows=3, cols=1,
                   subplot_titles=("Products Ordered",
                                   "Average Return Rate",
                                   "Total Spending"))

fig.append_trace(go.Histogram(x=customers.products_ordered),
                 row=1, col=1)

fig.append_trace(go.Histogram(x=customers.average_return_rate),
                 row=2, col=1)

fig.append_trace(go.Histogram(x=customers.total_spending),
                 row=3, col=1)

fig.update_layout(height=800, width=800,
                  title_text="Distribution of the Features")

fig.show()

## Scale Features: Log Transformation

In [23]:
def apply_log1p_transformation(dataframe, column):
    '''This function takes a dataframe and a column in the string format
    then applies numpy log1p transformation to the column
    as a result returns log1p applied pandas series'''

    dataframe["log_" + column] = np.log1p(dataframe[column])
    return dataframe["log_" + column]

### 1. Products ordered

In [24]:
apply_log1p_transformation(customers, "products_ordered")

0        0.693147
1        0.693147
2        0.693147
3        1.098612
4        0.693147
           ...   
16504    0.693147
16505    0.693147
16506    0.693147
16507    0.693147
16508    0.693147
Name: log_products_ordered, Length: 16509, dtype: float64

### 2. Average return rate

In [25]:
apply_log1p_transformation(customers, "average_return_rate")

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
16504    0.0
16505    0.0
16506    0.0
16507    0.0
16508    0.0
Name: log_average_return_rate, Length: 16509, dtype: float64

### 3. Total spending

In [26]:
apply_log1p_transformation(customers, "total_spending")

0        5.564520
1        4.384524
2        4.384524
3        4.644391
4        5.036953
           ...   
16504    4.330733
16505    4.867534
16506    4.787492
16507    4.460144
16508    4.174387
Name: log_total_spending, Length: 16509, dtype: float64

### Visualize log transformation applied features

In [27]:
fig = make_subplots(rows=3, cols=1,
                   subplot_titles=("Products Ordered",
                                   "Average Return Rate",
                                   "Total Spending"))

fig.append_trace(go.Histogram(x=customers.log_products_ordered),
                 row=1, col=1)

fig.append_trace(go.Histogram(x=customers.log_average_return_rate),
                 row=2, col=1)

fig.append_trace(go.Histogram(x=customers.log_total_spending),
                 row=3, col=1)

fig.update_layout(height=800, width=800,
                  title_text="Distribution of the Features after Logarithm Transformation")

fig.show()

In [28]:
customers.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending
0,1,0.0,260.0,0.693147,0.0,5.564520
1,1,0.0,79.2,0.693147,0.0,4.384524
2,1,0.0,79.2,0.693147,0.0,4.384524
3,2,0.0,103.0,1.098612,0.0,4.644391
4,1,0.0,153.0,0.693147,0.0,5.036953


In [29]:
# features we are going to use as an input to the model
customers.iloc[:,3:]

,log_products_ordered,log_average_return_rate,log_total_spending
0,0.693147,0.0,5.564520
1,0.693147,0.0,4.384524
2,0.693147,0.0,4.384524
3,1.098612,0.0,4.644391
4,0.693147,0.0,5.036953
...,...,...,...
16504,0.693147,0.0,4.330733
16505,0.693147,0.0,4.867534
16506,0.693147,0.0,4.787492
16507,0.693147,0.0,4.460144


## Create K-means model

In [30]:
# create initial K-means model
kmeans_model = KMeans(init='k-means++',
                      max_iter=500,
                      random_state=42)

In [31]:
kmeans_model.fit(customers.iloc[:,3:])

# print the sum of distances from all examples to the center of the cluster
print("within-cluster sum-of-squares (inertia) of the model is:", kmeans_model.inertia_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



within-cluster sum-of-squares (inertia) of the model is: 486.1457645961467


## Hyperparameter tuning: Find optimal number of clusters

In [32]:
def make_list_of_K(K, dataframe):
    '''inputs: K as integer and dataframe
    apply k-means clustering to dataframe
    and make a list of inertia values against 1 to K (inclusive)
    return the inertia values list
    '''

    cluster_values = list(range(1, K+1))
    inertia_values=[]

    for c in cluster_values:
        model = KMeans(
            n_clusters = c,
            init='k-means++',
            max_iter=500,
            random_state=42)
        model.fit(dataframe)
        inertia_values.append(model.inertia_)

    return inertia_values

### Visualize different K and models

In [33]:
# save inertia values in a dataframe for k values between 1 to 15
results = make_list_of_K(15, customers.iloc[:, 3:])

k_values_distances = pd.DataFrame({"clusters": list(range(1, 16)),
                                   "within cluster sum of squared distances": results})

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

In [34]:
# visualization for the selection of number of segments
fig = go.Figure()

fig.add_trace(go.Scatter(x=k_values_distances["clusters"],
                         y=k_values_distances["within cluster sum of squared distances"],
                         mode='lines+markers'))

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                  title_text="Within Cluster Sum of Squared Distances VS K Values",
                  xaxis_title="K values",
                  yaxis_title="Cluster sum of squared distances")

fig.show()

## Update K-Means Clustering

In [35]:
# create clustering model with optimal k=4
updated_kmeans_model = KMeans(n_clusters = 4,
                              init='k-means++',
                              max_iter=500,
                              random_state=42)

updated_kmeans_model.fit_predict(customers.iloc[:,3:])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([3, 2, 2, ..., 0, 2, 2], dtype=int32)

### Add cluster centers to the visualization

In [36]:
# create cluster centers and actual data arrays
cluster_centers = updated_kmeans_model.cluster_centers_
actual_data = np.expm1(cluster_centers)
add_points = np.append(actual_data, cluster_centers, axis=1)
add_points

array([[1.04785648e+00, 9.94811766e-05, 1.17187364e+02, 7.16793626e-01,
        9.94762287e-05, 4.77227119e+00],
       [3.03219840e+00, 5.37568100e-05, 3.91680505e+02, 1.39431173e+00,
        5.37553651e-05, 5.97299632e+00],
       [1.00536027e+00, 8.89099234e-04, 7.12945519e+01, 6.95823731e-01,
        8.88704220e-04, 4.28074877e+00],
       [1.85090153e+00, 3.21996642e-04, 1.89210879e+02, 1.04763527e+00,
        3.21944812e-04, 5.24813334e+00]])

In [37]:
# add labels to customers dataframe and add_points array
add_points = np.append(add_points, [[0], [1], [2], [3]], axis=1)
customers["clusters"] = updated_kmeans_model.labels_

In [38]:
# create centers dataframe from add_points
centers_df = pd.DataFrame(data=add_points, columns=["products_ordered",
                                                    "average_return_rate",
                                                    "total_spending",
                                                    "log_products_ordered",
                                                    "log_average_return_rate",
                                                    "log_total_spending",
                                                    "clusters"])
centers_df.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1.047856,0.000099,117.187364,0.716794,0.000099,4.772271,0.0
1,3.032198,0.000054,391.680505,1.394312,0.000054,5.972996,1.0
2,1.005360,0.000889,71.294552,0.695824,0.000889,4.280749,2.0
3,1.850902,0.000322,189.210879,1.047635,0.000322,5.248133,3.0


In [39]:
# align cluster centers of centers_df and customers
centers_df["clusters"] = centers_df["clusters"].astype("int")

In [40]:
centers_df.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1.047856,0.000099,117.187364,0.716794,0.000099,4.772271,0
1,3.032198,0.000054,391.680505,1.394312,0.000054,5.972996,1
2,1.005360,0.000889,71.294552,0.695824,0.000889,4.280749,2
3,1.850902,0.000322,189.210879,1.047635,0.000322,5.248133,3


In [41]:
customers.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1,0.0,260.0,0.693147,0.0,5.564520,3
1,1,0.0,79.2,0.693147,0.0,4.384524,2
2,1,0.0,79.2,0.693147,0.0,4.384524,2
3,2,0.0,103.0,1.098612,0.0,4.644391,0
4,1,0.0,153.0,0.693147,0.0,5.036953,0


In [42]:
# differentiate between data points and cluster centers
customers["is_center"] = 0
centers_df["is_center"] = 1

# add dataframes together
customers = customers.append(centers_df, ignore_index=True)

<ipython-input-42-1dcf0956e607>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [43]:
customers.tail()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters,is_center
16508,1.000000,0.000000,64.000000,0.693147,0.000000,4.174387,2,0
16509,1.047856,0.000099,117.187364,0.716794,0.000099,4.772271,0,1
16510,3.032198,0.000054,391.680505,1.394312,0.000054,5.972996,1,1
16511,1.005360,0.000889,71.294552,0.695824,0.000889,4.280749,2,1
16512,1.850902,0.000322,189.210879,1.047635,0.000322,5.248133,3,1


### Visualize Customer Segmentation

In [44]:
# add clusters to the dataframe
customers["cluster_name"] = customers["clusters"].astype(str)

In [45]:
# visualize log_transformation customer segments with a 3D plot
fig = px.scatter_3d(customers,
                    x="log_products_ordered",
                    y="log_average_return_rate",
                    z="log_total_spending",
                    color='cluster_name',
                    hover_data=["products_ordered",
                                "average_return_rate",
                                "total_spending"],
                    category_orders = {"cluster_name":
                                       ["0", "1", "2", "3"]},
                    symbol = "is_center"
                    )

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

## Check for Cluster Magnitude

In [46]:
# values for log_transformation
cardinality_df = pd.DataFrame(
    customers.cluster_name.value_counts().reset_index())

cardinality_df.rename(columns={"index": "Customer Groups",
                               "cluster_name": "Customer Group Magnitude"},
                      inplace=True)

In [47]:
cardinality_df

,Customer Groups,Customer Group Magnitude
0,2,6168
1,3,4306
2,0,4078
3,1,1961


In [48]:
fig = px.bar(cardinality_df, x="Customer Groups",
             y="Customer Group Magnitude",
             color = "Customer Groups",
             category_orders = {"Customer Groups": ["0", "1", "2", "3"]})

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                 yaxis = dict(
        tickmode = 'linear',
        tick0 = 1000,
        dtick = 1000))

fig.show()